In [ ]:
#Importação das bibliotecas
import pandas as pd
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

In [ ]:
#Importação da base de dados do salão
caminho = 'base_salao.xlsx' #Alocar a planilha na mesma pasta do script
df = pd.read_excel(caminho,sheet_name='AGENDA')

In [ ]:
#Formata o horário para remover os segundos da coluna HORA e o horário da coluna DATA
df['HORARIO'] = df['HORARIO'].apply(lambda x : x.strftime("%H:%M"))
df['DATA'] = df['DATA'].apply(lambda x : x.strftime("%d/%m/%y"))

In [ ]:
#Filtra o dataframe com base nos clientes que ainda precisam ser avisados
df_avisar = df.loc[df['AVISAR']=='SIM'].reset_index(drop=True)
df_avisar

In [ ]:
#Abre o navegador para logar o whatsapp
driver = webdriver.Chrome()
driver.get('https://web.whatsapp.com/')
sleep(10)

#Loop para logar o captcha
while True:
    try:
        campo_busca = driver.find_element(By.XPATH,'//*[@id="side"]/div[1]/div/div[2]/div[2]/div/div[1]')
        break #whatsapp já está logado

    except NoSuchElementException:
        print("Whatsapp ainda não foi logado, nova tentativa em 10 segundos")
        sleep(10)

In [ ]:
#laço para enviar a mensagem para todos do dataframe
for indice in df_avisar.index:

    info = df_avisar.iloc[indice] #obtem todas as informações do agendamento

    #Abre a conversa da pessoa
    campo_busca = driver.find_element(By.XPATH,'//*[@id="side"]/div[1]/div/div[2]/div[2]/div/div[1]')
    campo_busca.send_keys(str(info[5]))
    sleep(2)
    campo_busca.send_keys(Keys.ENTER)

    #Envia a mensagem
    campo_mensagem = driver.find_element(By.XPATH,'//*[@id="main"]/footer/div[1]/div/span[2]/div/div[2]/div[1]/div/div[1]/p')
    campo_mensagem.send_keys(f'Olá, {info[0]}!')
    sleep(1.5)
    campo_mensagem.send_keys(Keys.SHIFT + Keys.ENTER)
    sleep(1.5)
    campo_mensagem.send_keys(f'Estou passando para lembrar que seu agendamento com *{info[4]}* para *{info[3]}* está marcado para:')
    sleep(1.5)
    campo_mensagem.send_keys(Keys.SHIFT + Keys.ENTER)
    sleep(1.5)
    campo_mensagem.send_keys(f'*{info[1]}* as *{info[2]}*')
    sleep(1.5)
    campo_mensagem.send_keys(Keys.SHIFT + Keys.ENTER)
    sleep(1.5)
    campo_mensagem.send_keys('Aguardamos você! Caso precise cancelar, basta nos avisar.')
    sleep(1.5)
    campo_mensagem.send_keys(Keys.ENTER)
    sleep(2)